In [1]:
import asyncio
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()

True

<h3>GraphQL Query Creator</h3>

In [2]:


def generate_yelp_graphql_query(search_locations):
    """
    Generates a GraphQL query for the Yelp API based on a list of search locations.

    Parameters:
    - search_locations (list): A list of dictionaries, each containing 'name', 'address', 'city', 'state', and 'country' keys.

    Returns:
    - str: A GraphQL query string for the provided search locations.
    """
    query_parts = ["query MyQuery{"]
    for index, location in enumerate(search_locations):
        # Sanitize the name by removing spaces and special characters
        sanitized_name = ''.join(e for e in location['name'] if e.isalnum())
        unique_name = f"search{sanitized_name}{index}"
        query_parts.append(
            f"{unique_name}: business_match("
            f"address1: \"{location['address']}\", "
            f"city: \"{location['city']}\", "
            f"country: \"{location['country']}\", "
            f"name: \"{location['name']}\", "
            f"state: \"{location['state']}\""
            ") {"
            "  business {"
            "    name"
            "    photos"
            "    rating"
            "    coordinates {"
            "      latitude"
            "      longitude"
            "    }"
            "    location {"
            "      formatted_address"
            "    }"
            "  }"
            "}"
        )
    query_parts.append("}")
    query = " ".join(query_parts)
    return json.dumps({"query": query})

    # """
    # Generates a GraphQL query for the Yelp API based on a list of search locations.

    # Parameters:
    # - search_locations (list): A list of dictionaries, each containing 'name' and 'address' keys.

    # Returns:
    # - str: A GraphQL query string for the provided search locations.
    # """
    # query_parts = ["query MyQuery{"]
    # for index, location in enumerate(search_locations):
    #     print(location)
    #     unique_name = f"search{location['name'].replace(' ', '')}{index}"
    #     query_parts.append(
    #         f"{unique_name}: business_match(location: \"{location['address']}\", limit: 1) {{"
    #         "  business {"
    #         "    name"
    #         "    photos"
    #         "    rating"
    #         "    coordinates {"
    #         "      latitude"
    #         "      longitude"
    #         "    }"
    #         "    location {"
    #         "      formatted_address"
    #         "    }"
    #         "  }"
    #         "}"
    #     )
    # query_parts.append("}")
    # query = " ".join(query_parts)
    # return json.dumps({"query": query})

<h3>Yelp API Output Parser</h3>

In [3]:
def parse_yelp_response(response_text):
    """
    Parses the Yelp GraphQL API response and formats it into a list of dictionaries.

    Parameters:
    - response_text (str): The JSON response text from the Yelp API.

    Returns:
    - list: A list of dictionaries, each containing 'name', 'address', 'photo', 'rating', 'lat', and 'lon'.
    """
    print(response_text)
    data = json.loads(response_text).get('data', {})
    formatted_results = []

    for search_key, search_result in data.items():
        for business in search_result.get('business', []):
            formatted_results.append({
                'name': business.get('name'),
                'address': business['location']['formatted_address'].replace('\n', ' '),
                'photo': business['photos'][0] if business['photos'] else None,
                'rating': business.get('rating'),
                'lat': business['coordinates'].get('latitude'),
                'lon': business['coordinates'].get('longitude')
            })

    return formatted_results

<h3>Yelp API BATCH Wrapper</h3>

In [4]:
async def yelp_batch_wrapper(search_locations):
    """
    Fetches data from Yelp API asynchronously using GraphQL.

    Parameters:
    - search_locations (list): A list of dictionaries with 'name' and 'address' for search locations.
    - headers (dict): Headers to include in the request.

    Returns:
    - dict: Parsed JSON response from the Yelp API.
    """
    url = "https://api.yelp.com/v3/graphql"
    payload = generate_yelp_graphql_query(search_locations)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f"Bearer {os.getenv('YELP_API_KEY')}"
    }
    try:
        response = await asyncio.to_thread(requests.post, url, headers=headers, data=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
        return parse_yelp_response(response.text)
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except Exception as err:
        print(f"An error occurred: {err}")

<h3>Yelp API Location Wrapper</h3>

In [5]:
async def yelp_wrapper(location, term="attractions", radius=20000, sort_by="best_match", limit=20):
    # Get a list of locations from Yelp API
    params = {
        "location": location,
        "term": term,
        "radius": radius,
        "sort_by": sort_by,
        "limit": limit
    }
    yelp_url = f"https://api.yelp.com/v3/businesses/search?{urllib.parse.urlencode(params)}"
    yelp_header = {
        "accept": "application/json",
        "Authorization": f"Bearer {os.getenv('YELP_API_KEY')}"
    }
    response = await asyncio.to_thread(requests.get, yelp_url, headers=yelp_header)
    if response.status_code == 200:
        yelp_data = response.json()
        # Parse and format the response
        formatted_locations = [
            {
                "name": business.get("name"),
                "address": " ".join(business["location"].get("display_address", [])),
                "photo": business.get("image_url"),
                "rating": business.get("rating"),
                "description": ", ".join(category["title"] for category in business.get("categories", [])),
                "lat": business["coordinates"].get("latitude"),
                "lon": business["coordinates"].get("longitude")
            }
            for business in yelp_data.get("businesses", [])
        ]
        
        
    
        return json.dumps(formatted_locations)
    
    else:
        response.raise_for_status()

<h3>OpenAI Wrapper</h3>

In [6]:
from openai import AsyncOpenAI

async def openai_wrapper(location, preferences = ""):
    # Get a list of 10 locations form OpenAI api
    openai_client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    print(location, preferences)
    client_content = f"""
I am traveling to {location}, and I want to visit attractions around this area that fit these criteria: {preferences}.
Generate a list of 10 attraction objects and return in this format:
{{
    "attractions": [
        {{
            "name": "<name>",
            "address": "<street address>",
            "city": "<city>",
            "state": "<state>",
            "country": "<country>",
        }}
    ]
}}
"""
    chat_completion = await openai_client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful travel assistant designed to output JSON containing a list of objects that have atrribute name and address."},
            {"role": "user", "content": client_content}
        ],
    )
    return json.loads(chat_completion.choices[0].message.content)["attractions"]

<h3>Attractions Wrapper</h3>

In [8]:
async def attractions_wrapper(location, preferences=""):
    # Get results from OpenAI
    openai_result = await openai_wrapper(location, preferences)
    
    # Add a source field to OpenAI results
    # openai_result_with_source = [{**attraction, "source": "openai"} for attraction in openai_result]
    
    # Feed addresses from openai_result back into yelp_wrapper
    print(openai_result)
    yelp_results = await yelp_batch_wrapper(openai_result)
    
    print(yelp_results)
    # yelp_results_from_openai = await asyncio.gather(*yelp_tasks)
    
    # Parse the new Yelp results and mark them as from Yelp
    # yelp_results_from_openai_with_source = []
    # for yelp_result in yelp_results_from_openai:
    #     for attraction in json.loads(yelp_result):
    #         yelp_results_from_openai_with_source.append({**attraction, "source": "yelp"})
    
    # # Get initial Yelp results
    # initial_yelp_result = await yelp_wrapper(location, preferences)
    # initial_yelp_result_with_source = [{**attraction, "source": "yelp"} for attraction in json.loads(initial_yelp_result)]
    
    # # Combine all results
    # combined_results = openai_result_with_source + yelp_results_from_openai_with_source + initial_yelp_result_with_source
    
    # # Deduplicate combined results based on name and address
    # seen = set()
    # unique_results = []
    # for result in combined_results:
    #     identifier = (result['name'].lower(), result['address'].lower())
    #     if identifier not in seen:
    #         seen.add(identifier)
    #         unique_results.append(result)
    
    # return unique_results
# async def attractions_wrapper(location, preferences=""):
#     # Run both openai_wrapper and yelp_wrapper concurrently
#     openai_result, yelp_result = await asyncio.gather(
#         openai_wrapper(location, preferences),
#         yelp_wrapper(location, preferences)
#     )
    
#     print(openai_result)
#     # print(yelp_result)
#     return yelp_result
    
#     # return {"OpenAI": openai_result, "Yelp": yelp_result}

In [9]:
await attractions_wrapper("San Francisco, CA", "museum")

San Francisco, CA museum
[{'name': 'San Francisco Museum of Modern Art (SFMOMA)', 'address': '151 3rd St', 'city': 'San Francisco', 'state': 'CA', 'country': 'USA'}, {'name': 'California Academy of Sciences', 'address': '55 Music Concourse Dr', 'city': 'San Francisco', 'state': 'CA', 'country': 'USA'}, {'name': 'Exploratorium', 'address': 'Pier 15, The Embarcadero & Green St.', 'city': 'San Francisco', 'state': 'CA', 'country': 'USA'}, {'name': 'de Young Museum', 'address': '50 Hagiwara Tea Garden Dr', 'city': 'San Francisco', 'state': 'CA', 'country': 'USA'}, {'name': 'Asian Art Museum', 'address': '200 Larkin St', 'city': 'San Francisco', 'state': 'CA', 'country': 'USA'}, {'name': 'Museum of the African Diaspora', 'address': '685 Mission St', 'city': 'San Francisco', 'state': 'CA', 'country': 'USA'}, {'name': 'Contemporary Jewish Museum', 'address': '736 Mission St', 'city': 'San Francisco', 'state': 'CA', 'country': 'USA'}, {'name': 'Legion of Honor', 'address': '100 34th Ave', 'cit